In [23]:
import tensorflow as tf
print(tf.__version__)

2.8.2


In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , LSTM , Dropout , Embedding , Bidirectional
from tensorflow.keras.callbacks import ModelCheckpoint , EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import BatchNormalization

In [2]:
path = '/content/alice_in_wonderland.txt'
file1 = open(path,'r',encoding='utf-8').read()
file1 = file1.lower()
file1 = file1.split('\n')

In [3]:
tokenizer = Tokenizer()

In [4]:
tokenizer.fit_on_texts(file1)

In [5]:
total_words = len(tokenizer.word_index)+1

In [6]:
print(f'total words {total_words}')


total words 2647


In [7]:
file1[0]

"alice's adventures in wonderland"

In [8]:
tokenizer.texts_to_sequences(['i am good'])[0]

[10, 250, 166]

In [9]:
token_list = [10, 250, 166]

In [10]:
input_sequences = []
for file2 in file1:
  token_list = tokenizer.texts_to_sequences([file2])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [11]:
max_len = max([len(i)for i in input_sequences])

In [46]:
input_sequences[6] 

[1490, 1491]

In [13]:
padding = pad_sequences(input_sequences,maxlen=16,padding='pre')

In [17]:
input_sequences[0]

[278, 423]

In [18]:
padding[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0, 278, 423], dtype=int32)

In [19]:
padding.shape

(25073, 16)

In [14]:
X = padding[:,:-1]
y = padding[:,-1]

In [15]:
y = to_categorical(y,2647)

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=92)

In [24]:
X_test.shape

(3761, 15)

In [45]:
len(y[0])

2647

In [18]:
model = Sequential()
model.add(Embedding(total_words, 300, input_length=15))
model.add(Bidirectional(LSTM(350,return_sequences=True)))
model.add(Bidirectional(LSTM(350)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
# earlystop = EarlyStopping(monitor='accuracy', min_delta=0, patience=50, verbose=1, mode='max')
filepath="accuracy_nlp-next_word.h5"
checkpoint = ModelCheckpoint(filepath,save_best_only=True, mode='min')

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 300)           794100    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 700)          1822800   
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 700)              2942800   
 nal)                                                            
                                                                 
 dense (Dense)               (None, 2647)              1855547   
                                                                 
Total params: 7,415,247
Trainable params: 7,415,247
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(X, y, epochs=200, verbose=1,batch_size=1024 , callbacks=[checkpoint],validation_data=(X_test,y_test))

Epoch 1/200
25/25 [==============================] - 15s 206ms/step - loss: 6.4894 - accuracy: 0.0515 - val_loss: 5.9818 - val_accuracy: 0.0561
Epoch 2/200
25/25 [==============================] - 4s 155ms/step - loss: 5.9840 - accuracy: 0.0560 - val_loss: 5.9214 - val_accuracy: 0.0561
Epoch 3/200
25/25 [==============================] - 4s 157ms/step - loss: 5.9268 - accuracy: 0.0599 - val_loss: 5.8222 - val_accuracy: 0.0564
Epoch 4/200
25/25 [==============================] - 4s 156ms/step - loss: 5.7803 - accuracy: 0.0638 - val_loss: 5.6642 - val_accuracy: 0.0643
Epoch 5/200
25/25 [==============================] - 4s 157ms/step - loss: 5.6393 - accuracy: 0.0743 - val_loss: 5.5259 - val_accuracy: 0.0787
Epoch 6/200
25/25 [==============================] - 4s 158ms/step - loss: 5.5097 - accuracy: 0.0871 - val_loss: 5.4091 - val_accuracy: 0.0840
Epoch 7/200
25/25 [==============================] - 4s 160ms/step - loss: 5.3984 - accuracy: 0.0931 - val_loss: 5.3029 - val_accuracy: 0.090

In [21]:
from tensorflow.keras.models import load_model

In [22]:
model = load_model('/content/accuracy_nlp-next_word.h5')

In [53]:
text ="Alice could see, as well"

In [54]:
t_to_seq = tokenizer.texts_to_sequences([text])[0]

In [37]:
t_to_seq

[66, 2, 58, 11, 4, 51, 110]

In [50]:
padding = pad_sequences([t_to_seq],maxlen=15,padding='pre')

In [42]:
padding

array([[  0,   0,   0,   0,   0,   0,   0,   0,  66,   2,  58,  11,   4,
         51, 110]], dtype=int32)

In [43]:
# predicted = model.predict_classes(padding)
# x=np.argmax(predicted,axis=1)
predict = np.argmax(model.predict(padding), axis=-1)

In [32]:
print(predict)

[2]


In [64]:
tokenizer.word_index.items()

AttributeError: ignored

In [44]:
output = None
for word , index in tokenizer.word_index.items():
  if  index == predict:
    output = word
    break
print(output)

such


In [56]:
seed_text = ''
for _ in range(20):
    token_list = tokenizer.texts_to_sequences([text])[0]
    padding = pad_sequences([token_list],maxlen=15, padding='pre')
    predicted = predict = np.argmax(model.predict(padding), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    text += " " + output_word
print(text)

Alice could see, as well as if she were looking over their sleep she felt two if the caterpillar was there had quite the way it's in their names were in their table with the


In [ ]:
# In another moment down went Alice after it, never once
# considering how in the world she was to get out again.

In [65]:
embeddings = model.layers[0].get_weights()[0]
embeddings[60]

array([-0.08344252,  0.00510313, -0.17960808, -0.03472053, -0.10028249,
        0.0691199 , -0.03117628,  0.09111877,  0.03555365, -0.12193644,
       -0.10330232,  0.06596287,  0.03676869, -0.07250361, -0.03804107,
        0.05481109, -0.0142484 ,  0.21630506, -0.01342051,  0.1022603 ,
       -0.02084466,  0.00445812, -0.13012843,  0.02257405,  0.13256058,
        0.04733593, -0.06337781, -0.07571854,  0.06618419,  0.11430182,
       -0.07963777,  0.10212055, -0.02632232, -0.03299565,  0.0067985 ,
        0.07192999, -0.10378733, -0.13070299,  0.08883631, -0.13576822,
        0.03922619,  0.08726501,  0.00383673,  0.04288139, -0.04299189,
       -0.00259255,  0.13386863,  0.00549963, -0.18601252,  0.0454355 ,
        0.06126179, -0.00191975, -0.09429233,  0.00603326,  0.05206077,
       -0.11694278, -0.11521396, -0.05419602, -0.03972219,  0.12855276,
        0.05615821,  0.06818191,  0.07098094, -0.03255486,  0.00133745,
       -0.01245874,  0.04858655, -0.11350741,  0.1132172 , -0.13

In [ ]:
#https://stackoverflow.com/questions/51235118/how-to-get-word-vectors-from-keras-embedding-layer

In [112]:
em = {index : embeddings[index]  for word , index in tokenizer.word_index.items()}

In [113]:
len(em[1])

300

In [89]:
em['feel']

array([ 5.43014854e-02,  9.03913081e-02, -4.69314717e-02, -1.19568761e-02,
        3.50401960e-02,  4.88642789e-02, -5.29776216e-02,  2.47012135e-02,
        1.08202640e-02,  6.11497648e-02,  1.23267673e-01, -1.09883837e-01,
        2.59629115e-02, -7.69578665e-02,  1.10376161e-02, -4.87868823e-02,
        4.57184874e-02, -6.90467507e-02, -7.18417764e-02,  5.19953035e-02,
       -3.26016173e-02,  5.97062595e-02, -3.76308151e-02,  1.27445664e-02,
        1.18338346e-01, -1.08736888e-01, -2.06767842e-02, -9.19770729e-03,
        1.45420969e-01,  6.94157481e-02, -1.93400923e-02,  3.49471439e-03,
        2.00287580e-01,  9.43972990e-02,  1.56609908e-01, -7.89398849e-02,
        3.23599651e-02, -4.25755978e-02,  4.50995117e-02, -8.08374360e-02,
       -1.72068268e-01, -1.64860114e-02,  6.56236187e-02, -1.14026368e-01,
        9.23725665e-02, -9.16553214e-02,  2.13897619e-02, -6.64447770e-02,
        1.86054204e-02, -2.54221000e-02, -4.06147763e-02,  8.64942968e-02,
       -1.69790536e-01, -

In [73]:
from sklearn.metrics.pairwise import cosine_similarity

In [93]:
print(cosine_similarity([em['she']], [em['her']])[0][0])

0.1461379


In [117]:
def similarity(word):
  
  cos = [cosine_similarity([em[word]],[em[i+1]]) for i in range(len(em))]
  return sorted(cos,key=max)[0]

In [118]:
print(similarity(5))

[[-0.8028196]]
